In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
class GrayscaleCIFAR100(Dataset):
    def __init__(self,root='./data', train=True, transform=None):
        self.cifar100 = datasets.CIFAR100(root=root, train=train, download=True)
        self.transform = transform
        self.grayscale = transforms.Grayscale()

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, index):
        image, label = self.cifar100[index]
        
        # Convert the image to grayscale
        
        grayscale_image = self.grayscale(image)

        # Apply transformations if provided
        if self.transform is not None:
            grayscale_image = self.transform(grayscale_image)
            image = self.transform(image)
        
        return grayscale_image, image

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

In [ ]:
train_data = GrayscaleCIFAR100(
    root='./data',
    train=True,
    transform=transform
)

test_data = datasets.CIFAR100(
    root='./data',
    train=False,
    transform=transform
)


In [ ]:
train_data.transform = transform
test_data.transform = transform

In [ ]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images
data_iter = iter(train_loader)
grayscaled, colored = next(data_iter)


In [ ]:

# Plot the images
fig = plt.figure(figsize=(10, 5))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    gray = grayscaled[i] 
    print(gray.shape)
    gray = np.transpose(gray.numpy(), (1, 2, 0))
    ax.imshow(gray)
plt.show()

In [ ]:
# Plot the images
fig = plt.figure(figsize=(10, 5))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    color = colored[i] 
    print(color.shape)
    color = np.transpose(color.numpy(), (1, 2, 0))
    print(color.shape)
    ax.imshow(color)
plt.show()

In [ ]:
relu = nn.ReLU()

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # print(x.shape)
        x = self.conv1(x)
        x = relu(x)
        x = self.conv2(x)
        x = relu(x)
        return x


In [ ]:
class DecoderConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # output_size = (input_size - 1) * stride + kernel_size - 2 * padding
        self.up_conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2) 
        self.conv = DoubleConv(out_channels, out_channels)

    def forward(self, x, x1):
        # print(f'x shape: {x.shape}')
        # print(f'x1 shape: {x1.shape}')
        y1 = self.up_conv(x)
        # print(f'y1 shape: {y1.shape}')
        y2 = torch.cat((y1, x1))
        y = self.conv(y2)
        return y

In [ ]:
class UNet(nn.Module):
    def __init__(self, input_channels=1):
        super().__init__()

        # non trainable layers
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # trainable layers
        ## encoder 
        self.en_conv1 = DoubleConv(input_channels,32)
        self.en_conv2 = DoubleConv(32,64)
        self.en_conv3 = DoubleConv(64,128)
        self.en_conv4 = DoubleConv(128,256)
        
        ## bottleneck
        self.conv = DoubleConv(256,512)

        ## decoder
        self.de_conv4 = DecoderConv(512,256)
        self.de_conv3 = DecoderConv(256,128)
        self.de_conv2 = DecoderConv(128,64)
        self.de_conv1 = DecoderConv(64,32)
        
        ## reconstruct layer
        self.reconstruct = nn.Conv2d(32, 3, kernel_size=3)

    def forward(self, x):
        # encoder inference 
        # print(x.shape)
        e1 = self.en_conv1(x)
        p1 = self.pool(e1)
        # print(e1.shape)
        # print(p1.shape)
        
        e2 = self.en_conv2(p1)
        p2 = self.pool(e2)
        # print(e2.shape)
        # print(p2.shape)
        
        e3 = self.en_conv3(p2)
        p3 = self.pool(e3)
        # print(e3.shape)
        # print(p3.shape)
        
        e4 = self.en_conv4(p3)
        p4 = self.pool(e4)
        # print(e4.shape)
        # print(p4.shape)
        # bottleneck inference
        b = self.conv(p4)
        
        # decoder inference 
        d4 = self.de_conv4(b,e4)
        d3 = self.de_conv3(d4,e3)
        d2 = self.de_conv2(d3,e2)
        d1 = self.de_conv1(d2,e1)

        # reconstruct image
        colored_image = self.reconstruct(d1)


        
        return colored_image

In [16]:
from torch import optim
device = torch.device("cpu")
# Set up training parameters
lr = 0.001
num_epochs = 10

# Create data loaders
transform = transforms.Compose([
    transforms.ToTensor()
])

# Initialize your U-Net model
model = UNet()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train the model
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for grayscale_images, colored_images in train_loader:
        grayscale_images = grayscale_images.to(device)
        colored_images = colored_images.to(device)

        # print(grayscale_images.shape)

        optimizer.zero_grad()

        # Forward pass
        predicted_images = model(grayscale_images)

        # Compute the loss
        loss = criterion(predicted_images, colored_images)

        # Backpropagation and weight updates
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")


: 

: 